In [2]:
import pandas as pd
import numpy as np

path = '경로 지정'

In [5]:
keyword = pd.read_excel(path + '/노래 데이터 통합/song_keyword.xlsx',index_col=0) # 가사 전처리/ 가수 / 노래제목/ url 포함
lyrics = pd.read_excel(path + '/text 감정 추출 모델/lyrics_sentiment.xlsx',index_col=0) # 가사기반 감정 확률
song = pd.read_excel(path + '/audio feature 감정 추출 모델/해원 수정 후/book_ost_song_final_data (해원).xlsx',index_col=0) # audio_feature 기반 감정 확률

`song` : 전체 노래 DB + book_ost, af 기반 감정 확률 벡터 추출 결과

In [8]:
print(song.shape)
song.head()

(14091, 14)


,id,name,Artist,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,4MzcQtRM0xlUhs5Y0Quy1R,난 너만 있으면 되는데,Kwon Hyun Jung,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,48Jivh6jAj87eK6lj3FtUo,hoho,Hideyuki Hashimoto,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,1GKz0TpC7HktxQTirXpyZi,Setsu,Hideyuki Hashimoto,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,53BeSVY0mpmucCIr0pDkPp,If You Could See Me Now,Jon Belle,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,6JU2EiLrNcmPuzRYC89x5N,Arabesque (Debussy),Thomas Adès,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588


In [9]:
song.columns = ['id','name','Artist','empty_a', 'sadness_a', 'enthusiasm_a', 'worry_a', 'love_a', 'fun_a',
       'hate_a', 'happiness_a', 'boredom_a', 'relief_a', 'anger_a']
song.head()

,id,name,Artist,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,4MzcQtRM0xlUhs5Y0Quy1R,난 너만 있으면 되는데,Kwon Hyun Jung,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,48Jivh6jAj87eK6lj3FtUo,hoho,Hideyuki Hashimoto,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,1GKz0TpC7HktxQTirXpyZi,Setsu,Hideyuki Hashimoto,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,53BeSVY0mpmucCIr0pDkPp,If You Could See Me Now,Jon Belle,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,6JU2EiLrNcmPuzRYC89x5N,Arabesque (Debussy),Thomas Adès,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588


In [12]:
song_drop_index = song[song[['name','Artist']].duplicated()].index
song.drop(song_drop_index, inplace=True,axis=0)
song = song.reset_index().iloc[:,1:]
print(song.shape)
song

,id,name,Artist,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,4MzcQtRM0xlUhs5Y0Quy1R,난 너만 있으면 되는데,Kwon Hyun Jung,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,48Jivh6jAj87eK6lj3FtUo,hoho,Hideyuki Hashimoto,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,1GKz0TpC7HktxQTirXpyZi,Setsu,Hideyuki Hashimoto,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,53BeSVY0mpmucCIr0pDkPp,If You Could See Me Now,Jon Belle,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,6JU2EiLrNcmPuzRYC89x5N,Arabesque (Debussy),Thomas Adès,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11180,26Ow8vxj4n0uew1UP8CU56,Heart's Content - Strings Mix,Various Artists,0.999999,0.999999,0.000611,0.000276,0.999973,0.000354,0.000063,0.000703,0.000031,0.000216,0.000094
11181,0ZSgAdb1YrApzp72nA6IIw,comethru,Various Artists,0.923909,0.916362,0.000446,0.000514,0.946018,0.000537,0.044059,0.001276,0.000288,0.000416,0.052798
11182,5JL1HLtR9jrlQraBCT1DL9,We Don't Have To Take Our Clothes Off,Various Artists,0.000795,0.000816,0.000563,0.000167,0.003699,0.003605,0.000040,0.004658,0.999864,0.001423,0.000036
11183,7aObrOB2jVZJ2vJ79lQGjU,L.O.V.E.,Various Artists,0.002833,0.003284,0.018591,0.000122,0.045248,0.019038,0.002511,0.013638,0.925294,0.000500,0.003770


`lyrics` : 노래 DB 전체 가사 text 감정 확률 벡터 추출 결과

In [13]:
print(lyrics.shape)
lyrics.head()

(21027, 13)


,name,id,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737


In [14]:
lyrics_drop_index = lyrics[lyrics['id'].duplicated()].index
lyrics.drop(lyrics_drop_index,inplace=True, axis=0)
lyrics = lyrics.reset_index().iloc[:,1:]
print(lyrics.shape)
lyrics

(14020, 13)


,name,id,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14015,September Song - Guitar Acoustic,4FhQ9BYFO3yFAASazZdvRX,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
14016,Control,70RiiOJdklpSmLSBnKlcYF,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
14017,Heart's Content - Strings Mix,26Ow8vxj4n0uew1UP8CU56,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167
14018,We Don't Have To Take Our Clothes Off,5JL1HLtR9jrlQraBCT1DL9,0.006371,0.253928,0.007161,0.332610,0.181240,0.019137,0.057025,0.120555,0.000727,0.020817,0.000430


`keyword` : 가사 전처리/ 가수 / 노래제목/ url 포함

In [15]:
keyword = keyword[['url','id','name','Artist','key_word']]
print(keyword.shape)
keyword.head()

(21027, 5)


,url,id,name,Artist,key_word
0.0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...
1.0,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...
2.0,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...
3.0,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...
4.0,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...


In [16]:
keyword_drop_index = keyword[keyword['id'].duplicated()].index
keyword.drop(keyword_drop_index, inplace=True,axis=0)
keyword= keyword.reset_index().iloc[:,1:]
print(keyword.shape)
keyword

(14019, 5)


,url,id,name,Artist,key_word
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...
...,...,...,...,...,...
14014,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,4FhQ9BYFO3yFAASazZdvRX,September Song - Guitar Acoustic,Various Artists,innocent darling use talk til morning mixtape ...
14015,https://open.spotify.com/track/70RiiOJdklpSmLS...,70RiiOJdklpSmLSBnKlcYF,Control,Various Artists,wan na lose control even though old know shake...
14016,https://open.spotify.com/track/26Ow8vxj4n0uew1...,26Ow8vxj4n0uew1UP8CU56,Heart's Content - Strings Mix,Various Artists,love live without u one thing know sure love f...
14017,https://open.spotify.com/track/5JL1HLtR9jrlQra...,5JL1HLtR9jrlQraBCT1DL9,We Don't Have To Take Our Clothes Off,Various Artists,take clothes good time oh oh could dance party...


# 데이터 합치기

In [17]:
keyword.head()

,url,id,name,Artist,key_word
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...


In [18]:
lyrics.head()

,name,id,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737


In [19]:
song.head()

,id,name,Artist,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,4MzcQtRM0xlUhs5Y0Quy1R,난 너만 있으면 되는데,Kwon Hyun Jung,0.275256,0.294684,0.067680,0.414073,0.329284,0.257404,0.002675,0.571722,0.070783,0.257363,0.004994
1,48Jivh6jAj87eK6lj3FtUo,hoho,Hideyuki Hashimoto,0.313797,0.360171,0.074607,0.525126,0.243545,0.263902,0.002605,0.526271,0.087957,0.375637,0.004942
2,1GKz0TpC7HktxQTirXpyZi,Setsu,Hideyuki Hashimoto,0.294603,0.315632,0.071614,0.449824,0.322634,0.305093,0.002567,0.619334,0.139616,0.459778,0.005099
3,53BeSVY0mpmucCIr0pDkPp,If You Could See Me Now,Jon Belle,0.301516,0.378358,0.069185,0.546034,0.195756,0.218471,0.001646,0.429420,0.062893,0.302185,0.003287
4,6JU2EiLrNcmPuzRYC89x5N,Arabesque (Debussy),Thomas Adès,0.529414,0.641548,0.067014,0.750942,0.120171,0.141197,0.001567,0.266672,0.041860,0.179600,0.004588


In [20]:
song_final_data = pd.merge(keyword, lyrics.iloc[:,1:], how='inner', on='id')
song_final_data

,url,id,name,Artist,key_word,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14014,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,4FhQ9BYFO3yFAASazZdvRX,September Song - Guitar Acoustic,Various Artists,innocent darling use talk til morning mixtape ...,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
14015,https://open.spotify.com/track/70RiiOJdklpSmLS...,70RiiOJdklpSmLSBnKlcYF,Control,Various Artists,wan na lose control even though old know shake...,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
14016,https://open.spotify.com/track/26Ow8vxj4n0uew1...,26Ow8vxj4n0uew1UP8CU56,Heart's Content - Strings Mix,Various Artists,love live without u one thing know sure love f...,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167
14017,https://open.spotify.com/track/5JL1HLtR9jrlQra...,5JL1HLtR9jrlQraBCT1DL9,We Don't Have To Take Our Clothes Off,Various Artists,take clothes good time oh oh could dance party...,0.006371,0.253928,0.007161,0.332610,0.181240,0.019137,0.057025,0.120555,0.000727,0.020817,0.000430


In [21]:
song_final_data.columns

Index(['url', 'id', 'name', 'Artist', 'key_word', 'empty', 'sadness',
       'enthusiasm', 'worry', 'love', 'fun', 'hate', 'happiness', 'boredom',
       'relief', 'anger'],
      dtype='object')

In [22]:
song_final_data_1 = pd.merge(song_final_data, song,how='inner', on=['id','name','Artist'])
song_final_data_1

,url,id,name,Artist,key_word,empty,sadness,enthusiasm,worry,love,...,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...,0.016461,0.265974,0.067839,0.144616,0.116033,...,0.001468,0.000573,0.999999,0.000172,0.000438,0.000084,0.000297,0.000171,0.000331,0.000293
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...,0.122187,0.059084,0.039108,0.113228,0.024429,...,0.000728,0.000677,0.000237,0.001199,0.001109,0.000269,0.000683,0.999826,0.000080,0.000237
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...,0.005816,0.248961,0.003242,0.483580,0.044740,...,0.000805,0.000329,0.000452,0.000202,0.000271,0.000053,0.000246,0.000452,0.999978,0.000072
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...,0.077518,0.249271,0.011364,0.406069,0.011648,...,0.002584,0.000131,0.000329,0.002033,0.000132,0.000044,0.000142,0.000081,0.999633,0.000056
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...,0.001735,0.235752,0.001410,0.463885,0.142529,...,0.000332,0.000241,0.000060,0.000185,0.000300,1.000000,0.000386,0.000421,0.000141,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,https://open.spotify.com/track/74xCxQy50bkpvOJ...,74xCxQy50bkpvOJm5MgD5P,Murder in the City,Various Artists,get murder city revenge name one person dead p...,0.011245,0.162807,0.006966,0.456847,0.082303,...,1.000000,0.000819,0.000420,0.999982,0.000613,0.000018,0.001577,0.000052,0.000307,0.000015
8516,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,4FhQ9BYFO3yFAASazZdvRX,September Song - Guitar Acoustic,Various Artists,innocent darling use talk til morning mixtape ...,0.021068,0.348507,0.035223,0.432478,0.046740,...,0.016190,0.000213,0.000669,0.022570,0.000219,0.992936,0.000332,0.000298,0.000344,0.992026
8517,https://open.spotify.com/track/70RiiOJdklpSmLS...,70RiiOJdklpSmLSBnKlcYF,Control,Various Artists,wan na lose control even though old know shake...,0.002924,0.170607,0.004646,0.211214,0.003779,...,0.999998,0.000799,0.000706,0.999953,0.000468,0.000105,0.000734,0.000042,0.000216,0.000202
8518,https://open.spotify.com/track/26Ow8vxj4n0uew1...,26Ow8vxj4n0uew1UP8CU56,Heart's Content - Strings Mix,Various Artists,love live without u one thing know sure love f...,0.004582,0.297325,0.004344,0.583962,0.050943,...,0.999999,0.000611,0.000276,0.999973,0.000354,0.000063,0.000703,0.000031,0.000216,0.000094


In [23]:
song_final_data_1['url'].isnull().sum()

0

In [24]:
song_final_data_1.shape

(8520, 27)

In [26]:
song_final_data_1.to_excel(path + '/audio feature 감정 추출 모델/해원 수정 후/final_data 해원.xlsx')